<a href="https://colab.research.google.com/github/mortymh/moryton/blob/master/Titanic%20Survival%20Prediction%20with%20Logistic%20Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#ایمپورت کتابخانه‌ها و بارگذاری دیتا
'''
pandas (pd): کتابخانه اصلی برای کار با داده‌ها (DataFrame)

train_test_split: برای تقسیم داده به دو بخش آموزش و تست

LogisticRegression: مدل یادگیری ماشین برای طبقه‌بندی دودویی (مثلاً زنده مانده یا نه)

classification_report و confusion_matrix: برای ارزیابی مدل

OneHotEncoder: (تو این کد از pd.get_dummies استفاده کردیم که مشابه است) برای تبدیل داده‌های دسته‌ای به عددی

SimpleImputer: برای پر کردن مقادیر گمشده در داده‌ها
'''

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer


In [9]:
''' این سلول ابتدا گوگل درایو شما را به محیط کولب متصل می‌کند.
 وقتی این کد را اجرا می‌کنید، یک لینک و یک کد تایید نمایش داده می‌شود که باید آن را وارد کنید
 تا اجازه دسترسی کولب به فایل‌های گوگل درایوتان داده شود.
 پس از مونت شدن درایو، فایل دیتاست Titanic که قبلاً در مسیر مشخص ذخیره کرده‌ایم
 به کمک pandas خوانده شده و پنج ردیف اول آن نمایش داده می‌شود.
 این روش باعث می‌شود که نیازی به دانلود مجدد داده‌ها نداشته باشید
 و همیشه بتوانید سریعاً به دیتاست از گوگل درایو دسترسی داشته باشید.
 '''


from google.colab import drive
drive.mount('/content/drive')

import pandas as pd

file_path = '/content/drive/MyDrive/titanic.csv'  # مسیر فایل در درایو

df = pd.read_csv(file_path)
print(df.head())


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9

In [10]:
#انتخاب ویژگی‌ها و متغیر هدف
'''
لیستی از ستون‌هایی که به مدل می‌دهیم را مشخص می‌کنیم (متغیرهای پیش‌بینی کننده یا Featureها)

X شامل داده ورودی (ویژگی‌ها) است

y متغیر هدف (Target) است که می‌خواهیم پیش‌بینی کنیم — بقا یا عدم بقا (0 یا 1)
'''
features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']
X = df[features]
y = df['Survived']


In [12]:
# پر کردن مقادیر گمشده در ستون Age
'''
ستون Age تعدادی مقدار خالی (NaN) دارد که مدل نمی‌تواند آن را تحلیل کند.
SimpleImputer مقدارهای خالی را با میانگین ستون Age جایگزین می‌کند (strategy='mean').
fit_transform هم ابتدا میانگین را محاسبه می‌کند (fit) و سپس مقدارهای خالی را جایگزین می‌کند (transform).
برای جلوگیری از هشدار SettingWithCopyWarning از روش .loc استفاده می‌کنیم.
همچنین خروجی fit_transform آرایه‌ای دو بعدی است که با ravel به آرایه‌ی یک‌بعدی تبدیل می‌شود.
'''

from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='mean')

# جایگزینی مقادیر NaN ستون Age با میانگین آن ستون به صورت ایمن و بدون هشدار
X.loc[:, 'Age'] = imputer.fit_transform(X[['Age']]).ravel()


In [13]:
#تبدیل داده‌های دسته‌ای به عددی (One-Hot Encoding)
'''ستون‌های Sex و Embarked داده‌های متنی دارند (مثلاً Male/Female یا C/S/Q)

مدل‌های ML فقط عدد می‌خوانند، بنابراین این‌ها باید به اعداد تبدیل شوند

get_dummies هر دسته را به یک ستون باینری تبدیل می‌کند

drop_first=True باعث می‌شود یکی از ستون‌های تولید شده حذف شود تا از چندکولینیاری (multicollinearity) جلوگیری کنیم
'''
X = pd.get_dummies(X, columns=['Sex', 'Embarked'], drop_first=True)


In [14]:
#تقسیم داده به دو بخش آموزش و تست
'''داده‌ها به دو قسمت تقسیم می‌شوند:

۸۰٪ برای آموزش مدل

۲۰٪ برای ارزیابی مدل (تست)

random_state=42 برای تکرارپذیری تقسیم داده است (هر بار همین تقسیم انجام شود)
'''

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [15]:
#ساخت مدل و آموزش
'''
مدل رگرسیون لجستیک ساخته می‌شود

max_iter=200 یعنی حداکثر ۲۰۰ بار الگوریتم سعی می‌کند ضرایب را بهینه کند (اگر لازم باشد بیشتر هم می‌شود)

fit مدل را روی داده‌های آموزش (X_train و y_train) آموزش می‌دهد
'''
model = LogisticRegression(max_iter=200)
model.fit(X_train, y_train)


LogisticRegression(max_iter=200)

In [16]:
#پیش‌بینی روی داده تست و ارزیابی مدل
'''
predict با مدل آموزش دیده، خروجی را برای داده تست پیش‌بینی می‌کند

classification_report معیارهای دقت، دقت مثبت، یادآوری و F1-score را نشان می‌دهد

confusion_matrix جدول درستی و اشتباه پیش‌بینی‌های مدل را نشان می‌دهد (TP, FP, TN, FN)
'''
y_pred = model.predict(X_test)

from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))



              precision    recall  f1-score   support

           0       0.83      0.86      0.84       105
           1       0.79      0.74      0.76        74

    accuracy                           0.81       179
   macro avg       0.81      0.80      0.80       179
weighted avg       0.81      0.81      0.81       179

[[90 15]
 [19 55]]
